In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [2]:
training_file = open("/home/iradspm/Msc/NLP/dataset/kikamba_hmm_corpus")
all_lines = training_file.readlines()
training_file.close()

In [3]:
'''
Remove double spaces
fin = open("/home/iradspm/Msc/NLP/dataset/kikamba_bert_corpus","rt")
fout = open("/home/iradspm/Msc/NLP/dataset/kikamba_hmm_corpus", "wt")

for line in fin:
    fout.write(' '.join(line.split()))
fin.close()
fout.close()
'''

'\nRemove double spaces\nfin = open("/home/iradspm/Msc/NLP/dataset/kikamba_bert_corpus","rt")\nfout = open("/home/iradspm/Msc/NLP/dataset/kikamba_hmm_corpus", "wt")\n\nfor line in fin:\n    fout.write(\' \'.join(line.split()))\nfin.close()\nfout.close()\n'

In [4]:
import re
def split_words():
    for words in all_lines:
        clean_data = re.sub(r'\n','', words)
        split_words=clean_data.split(' ')
    return split_words

In [5]:
tagged=list(split_words())
tagged

['Tene/JJ',
 'yĩla/RB',
 'Ĩsilaeli/NP',
 "yatongoew'e/VB",
 'nĩ/PRE',
 'Asili/NNS',
 ',/COMMA',
 'nĩkweethĩiwe/VB',
 'na/CONJ',
 'yũa/NN',
 'nthĩ/NN',
 'ĩsu/JJ',
 './F-STOP',
 'Kwoou/JJ',
 'mũndũ/NN',
 'ũmwe/JJ',
 'kuma/PRE',
 'Mbetheleemu/NP',
 'nthĩ/NN',
 'ya/PRE',
 'Yuta/NP',
 'nĩwaendie/VB',
 'e/RB',
 'na/CONJ',
 'mũka/NN',
 'na/CONJ',
 'ana/NNS',
 'make/PP$',
 'elĩ/NUM',
 'kwĩkala/VB',
 'kwa/RB',
 'kavinda/NN',
 'ũeninĩ/JJ',
 'nthĩ/NN',
 'ya/PRE',
 'Moavi/NP',
 './F-STOP',
 'Mũndũ/NN',
 'ũsu/JJ',
 'eetawa/VB',
 'Elimeleki/NP',
 ',/COMMA',
 'na/CONJ',
 'mũka/NN',
 'eetawa/VB',
 'Naũmi/NP',
 './F-STOP',
 'Ana/NNS',
 'make/PP$',
 'ũmwe/NUM',
 'eetawa/VB',
 'Maloni/NP',
 'na/CONJ',
 'ũla/DET',
 'ũngĩ/JJ',
 'eetawa/VB',
 'Kilioni/NP',
 './F-STOP',
 'Andũ/NNS',
 'asu/JJ',
 'maĩ/VB',
 'ma/PP$',
 'mũsyĩ/NN',
 'wa/PRE',
 'Aevilathi/NPS',
 'ala/JJ',
 'matwĩe/VB',
 'Mbetheleemu/NP',
 'nthĩ/NN',
 'ya/PRE',
 'Yuta/NP',
 './F-STOP',
 'Nĩmaendie/VB',
 'Moavi/NP',
 'matũa/VB',
 "kw'o/RB",
 './F-S

In [6]:
nltk_data=[nltk.tag.str2tuple(t) for t in tagged]

In [7]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.90,test_size=0.10,random_state = 101)

In [8]:
train_set[:5]

[('ĩndĩ', 'RB'),
 ('maĩ', 'RB'),
 ('nĩweeyũmbanisye', 'VB'),
 ('na', 'CONJ'),
 ('.', 'F-STOP')]

In [9]:
# create list of train and test tagged words
train_tagged_words = list(train_set)
test_tagged_words = list(test_set)
print(len(train_tagged_words))
print(len(test_tagged_words))

2925
326


In [10]:
train_tagged_words[:5]

[('ĩndĩ', 'RB'),
 ('maĩ', 'RB'),
 ('nĩweeyũmbanisye', 'VB'),
 ('na', 'CONJ'),
 ('.', 'F-STOP')]

In [11]:
test_tagged_words[:5]

[(',', 'COMMA'), ('”', 'PUNC'), ('ũsu', 'JJ'), ('andũ', 'NNS'), (',', 'COMMA')]

In [12]:
tags_1= {tag for word,tag in train_tagged_words}

In [13]:
#tags=filter(lambda k: not isnan(k), tags_1)

In [14]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

18
{'PP$', 'NUM', 'PRE', 'NN', 'INJ', 'COMMA', 'VB', 'NNS', 'DET', 'NPS', None, 'CONJ', 'JJ', 'PUNC', 'PP', 'F-STOP', 'NP', 'RB'}


In [15]:
#vocab

In [16]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [17]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [18]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[0.06603774 0.02830189 0.04716981 0.13207547 0.         0.08490566
  0.20754717 0.02830189 0.00943396 0.         0.         0.06603774
  0.11320755 0.04716981 0.         0.02830189 0.06603774 0.0754717 ]
 [0.         0.         0.07692308 0.23076923 0.         0.15384616
  0.15384616 0.         0.07692308 0.         0.         0.
  0.23076923 0.         0.         0.07692308 0.         0.        ]
 [0.02793296 0.         0.06703911 0.12849163 0.         0.03351955
  0.26815644 0.01117318 0.00558659 0.         0.         0.06145252
  0.10055866 0.03351955 0.02234637 0.06703911 0.08379889 0.08938547]
 [0.05585106 0.         0.05585106 0.16489361 0.00265957 0.06914894
  0.1888298  0.03457447 0.00531915 0.         0.         0.0930851
  0.06382979 0.05053192 0.0106383  0.03989362 0.08244681 0.08244681]
 [0.         0.         0.         0.         0.         0.25
  0.         0.         0.         0.         0.         0.5
  0.         0.         0.         0.         0.         0.25     

In [19]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,PP$,NUM,PRE,NN,INJ,COMMA,VB,NNS,DET,NPS,NaN,CONJ,JJ,PUNC,PP,F-STOP,NP,RB
PP$,0.066038,0.028302,0.047170,0.132075,0.000000,0.084906,0.207547,0.028302,0.009434,0.000000,0.000000,0.066038,0.113208,0.047170,0.000000,0.028302,0.066038,0.075472
NUM,0.000000,0.000000,0.076923,0.230769,0.000000,0.153846,0.153846,0.000000,0.076923,0.000000,0.000000,0.000000,0.230769,0.000000,0.000000,0.076923,0.000000,0.000000
PRE,0.027933,0.000000,0.067039,0.128492,0.000000,0.033520,0.268156,0.011173,0.005587,0.000000,0.000000,0.061453,0.100559,0.033520,0.022346,0.067039,0.083799,0.089385
NN,0.055851,0.000000,0.055851,0.164894,0.002660,0.069149,0.188830,0.034574,0.005319,0.000000,0.000000,0.093085,0.063830,0.050532,0.010638,0.039894,0.082447,0.082447
INJ,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
COMMA,0.032609,0.000000,0.086957,0.146739,0.005435,0.043478,0.233696,0.054348,0.010870,0.000000,0.000000,0.043478,0.065217,0.038043,0.005435,0.043478,0.081522,0.108696
VB,0.038521,0.010786,0.052388,0.121726,0.001541,0.047766,0.240370,0.036980,0.003082,0.000000,0.000000,0.080123,0.097072,0.041602,0.007704,0.061633,0.069337,0.087827
NNS,0.032787,0.000000,0.032787,0.106557,0.000000,0.065574,0.196721,0.049180,0.000000,0.008197,0.000000,0.098361,0.073770,0.057377,0.016393,0.057377,0.065574,0.139344
DET,0.000000,0.000000,0.142857,0.142857,0.000000,0.071429,0.142857,0.142857,0.000000,0.000000,0.000000,0.071429,0.071429,0.000000,0.000000,0.000000,0.071429,0.142857
NPS,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['F-STOP', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [21]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
#test_run = [test_set[i] for i in rndom]
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base=test_run
#test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
#[pair[1] for pair in tagged_sents]
test_tagged_words = [tup[0] for tup in test_run]

In [22]:
rndom

[226, 60, 4, 47, 299, 18, 43, 51, 182, 122]

In [23]:
test_run

[('syathela', 'RB'),
 ('itiĩtwe', 'VB'),
 (',', 'COMMA'),
 ('athũkũmi', 'NNS'),
 ('wĩa', 'NN'),
 ('amwĩa', 'VB'),
 ('katĩ', 'JJ'),
 ('atĩĩ', 'JJ'),
 ('.', 'F-STOP'),
 ('nĩ', 'VB')]

In [24]:
test_tagged_words

['syathela',
 'itiĩtwe',
 ',',
 'athũkũmi',
 'wĩa',
 'amwĩa',
 'katĩ',
 'atĩĩ',
 '.',
 'nĩ']

In [25]:
test_run_base

[('syathela', 'RB'),
 ('itiĩtwe', 'VB'),
 (',', 'COMMA'),
 ('athũkũmi', 'NNS'),
 ('wĩa', 'NN'),
 ('amwĩa', 'VB'),
 ('katĩ', 'JJ'),
 ('atĩĩ', 'JJ'),
 ('.', 'F-STOP'),
 ('nĩ', 'VB')]

In [26]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  0.0902256965637207
Viterbi Algorithm Accuracy:  50.0


In [27]:
test_sent=" Amanyĩsya ma ũvũngũ wĩa woo nĩ kũnyungunyĩsya na kũlea kwĩanĩwa . "
pred_tags= Viterbi(test_sent.split())
print(pred_tags)

[('Amanyĩsya', 'NNS'), ('ma', 'PP$'), ('ũvũngũ', 'PP$'), ('wĩa', 'NN'), ('woo', 'JJ'), ('nĩ', 'PRE'), ('kũnyungunyĩsya', 'VB'), ('na', 'CONJ'), ('kũlea', 'VB'), ('kwĩanĩwa', 'VB'), ('.', 'F-STOP')]


In [28]:
#pip install gradio

In [29]:
import gradio as gr

In [30]:
def predictPos(sent_words):
    pred_tags= Viterbi(sent_words.split())
    return pred_tags

In [31]:
input_words = gr.inputs.Textbox(placeholder="Enter a sentence to predict POS")
output = gr.outputs.Textbox() 
#create interface
gui = gr.Interface(fn=predictPos,
                   inputs=[input_words],
                   outputs=[output])

/home/iradspm/anaconda3/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/iradspm/anaconda3/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/iradspm/anaconda3/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [32]:
gui.launch()

Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7ff269b727c0>, 'http://127.0.0.1:7861/', None)